In [1]:
import pandas as pd
import requests
import boto3    
import scrapy, os, logging
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go

from sqlalchemy import create_engine
import psycopg2

# If you are on the workspaces:
# pio.renderers.default = "iframe_connected"
# Animation works only in browser renderer
pio.renderers.default = "browser"

In [2]:

cities = ["Mont Saint Michel", "St Malo", "Bayeux", "Le Havre", "Rouen", "Paris", "Amiens", "Lille",
"Strasbourg", "Chateau du Haut Koenigsbourg", "Colmar", "Eguisheim", "Besancon", "Dijon", "Annecy",
"Grenoble", "Lyon", "Gorges du Verdon", "Bormes les Mimosas", "Cassis", "Marseille", "Aix en Provence",
"Avignon", "Uzes", "Nimes", "Aigues Mortes", "Saintes Maries de la mer", "Collioure","Carcassonne",
"Ariege", "Toulouse", "Montauban", "Biarritz", "Bayonne", "La Rochelle"]

In [6]:
# build dataframe incremetally
df = pd.DataFrame()
df['City'] = cities
df["pkey"] = df.index + 1
df.head()

,City,pkey
0,Mont Saint Michel,1
1,St Malo,2
2,Bayeux,3
3,Le Havre,4
4,Rouen,5


In [ ]:
# #######
# get GPS coordinate for each city
###
URL_GEOCODE = 'https://nominatim.openstreetmap.org/search?q={}&countrycodes=fr&limit=1&format=geocodejson'

dt_temp = pd.DataFrame()
for i, city in enumerate(cities):
    try :
        print("About to get GPS coordinate for : ", city, end=" ")
        results = requests.get(URL_GEOCODE.format(city)).json()
        latlong = results.get('features')[0].get('geometry').get('coordinates')
        df.loc[i,'lat'] = latlong[1]
        df.loc[i,'lon'] = latlong[0]
        print(" done.")
    except:
        print("Failed to get GPS coordinates for : ", city)
        print("Error is  :", results)
        df.loc[i,'lat'] = 0
        df.loc[i,'lon'] = 0

# let's check the result
df.head()

In [ ]:
# #####
# Now we go and get the weather for a week (only temp)
#####
API_KEY='8839888b57115cb1310677c08eb93a52'
URL_WEATHER='https://api.openweathermap.org/data/2.5/onecall?lat={}&lon={}4&appid=8839888b57115cb1310677c08eb93a52&units=metric&exclude=current,minutely,hourly'

# get the weather forecast for the next 7 days
temp_labels = ['Temp0', 'Temp1', 'Temp2', 'Temp3', 'Temp4', 'Temp5', 'Temp6']
for i, city in enumerate(cities):
    results = requests.get(URL_WEATHER.format(df.loc[i,'lat'], df.loc[i,'lon']))

    temp7days = []
    for d in range(len(temp_labels)):
       df.loc[i, temp_labels[d]]= results.json().get('daily')[d].get('temp').get('day')
df.head()

,City,pkey,lat,lon,Temp0,Temp1,Temp2,Temp3,Temp4,Temp5,Temp6
0,Mont Saint Michel,1,48.635954,-1.511460,5.30,8.36,7.22,6.35,7.48,8.22,7.22
1,St Malo,2,48.649518,-2.026041,4.87,8.72,7.35,6.16,7.29,7.32,6.21
2,Bayeux,3,49.276462,-0.702474,3.78,8.28,6.66,6.11,8.16,8.08,6.20
3,Le Havre,4,49.493898,0.107973,4.21,8.21,6.01,6.03,8.16,6.94,6.33
4,Rouen,5,49.440459,1.093966,5.64,8.25,5.02,5.31,8.33,7.13,7.63


In [70]:
# ######
# Let's save the result locally and on S3
# don't need index, we have a primary key
df.to_csv('city-info.csv', index=False)
s3 = boto3.resource('s3')
my_bucket = s3.Bucket('ycakayak')
my_bucket.put_object(Key="city-info.csv", Body=df.to_csv())

s3.Object(bucket_name='ycakayak', key='city-info.csv')

In [ ]:
#####
# Here we go scrap booking.com to the hotel for each city
# Data is saved locally as city-hotels.csv
################
class BookingSpider(scrapy.Spider):
    name = "booking"
    
    def __init__(self, start_urls=None, *args, **kwargs):
        self.start_urls = start_urls
        self.fkey = 0
        super(BookingSpider, self).__init__(*args, **kwargs)
    
    def parse(self, response):
        # get the list of hotels
        hotels = response.css('div._fe1927d9e')
        self.fkey +=1

        for hotel in hotels:
            try :
                name_hotel = hotel.css('div.fde444d7ef::text').get()
                href_hotel = hotel.css('h3._23bf57b84 a').attrib['href']
                desc_hotel = hotel.css('div._4abc4c3d5::text').get()
                note_hotel = hotel.css('div._9c5f726ff::text').get().replace(",", ".")
            except:
                print ("*** Error while processing : ", hotel)
            else:
                # then call the hotel page to get detailed info for each hotel
                yield scrapy.Request(
                    href_hotel, 
                    callback=self.hotel_page,
                    meta={
                        'fkey_hotel':self.fkey, 
                        'name_hotel':name_hotel, 
                        'desc_hotel':desc_hotel, 
                        'note_hotel':note_hotel, 
                        'href_hotel':href_hotel}
                    )
        
    def hotel_page(self, response):
        latlon = response.css('a.show_on_map_hp_link').attrib['data-atlas-latlng'].split(",")
        output = { 
            'fkey' : response.meta.get('fkey_hotel'),
            'name' : response.meta.get('name_hotel'),
            'desc' : response.meta.get('desc_hotel'),
            'note' : response.meta.get('note_hotel'),
            'href' : response.meta.get('href_hotel'),
            'lat' : latlon[0],
            'lon' : latlon[1]
        }
        return output
        
######
#
filename = 'city-hotels.json'
if filename in os.listdir('./'):
    os.remove(filename)

process = scrapy.crawler.CrawlerProcess(settings = {
    'USER_AGENT': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:15.0) Gecko/20100101 Firefox/15.0.1',
    'LOG_LEVEL': logging.INFO,
    "FEEDS": {
        filename : {"format": "json", "encoding": "utf8"}
    }
})

start_urls=[]
for city in cities:
    start_urls.append('https://www.booking.com/searchresults.fr.html?ss={}&nflt=ht_id%3D204'.format(city))

process.crawl(BookingSpider, start_urls=start_urls)
process.start()

In [38]:
####
# Load the result & save it to s3
##########
df = pd.read_csv('city-hotels.csv')

s3 = boto3.resource('s3')
my_bucket = s3.Bucket('ycakayak')
my_bucket.put_object(Key="city-hotels.csv", Body=df.to_csv())

s3.Object(bucket_name='ycakayak', key='city-hotels.csv')

In [3]:
"""
Master username : postgres
Master password : postgres
Endpoint : database-1.cq65oghuf9ur.eu-west-3.rds.amazonaws.com
"""

DATABASE = 'postgres'
TYPE_DB = 'postgresql+psycopg2'
USER_DB = 'postgres'
PASS_DB = 'postgres'
HOST_DB = 'database-1.cq65oghuf9ur.eu-west-3.rds.amazonaws.com'

conn = psycopg2.connect(
    database=DATABASE,
    user=USER_DB,
    password=PASS_DB,
    host=HOST_DB,
    port='5432'
)
engine = create_engine(TYPE_DB+'://'+USER_DB+':'+PASS_DB+'@'+HOST_DB, echo=True) 

In [ ]:
###
# Load data from s3 and create/update it to db
#############
s3 = boto3.client('s3')
s3.download_file('ycakayak', 'city-info.csv', 'city-info.csv')
df_cities = pd.read_csv('city-info.csv')
df_cities.to_sql('City', engine, if_exists="replace", index=False)

In [ ]:
###
# Load data from s3 and create/update it to db
#############
s3 = boto3.client('s3')
s3.download_file('ycakayak', 'city-hotels.csv', 'city-hotels.csv')
df_hotels = pd.read_csv('city-hotels.csv')
df_hotels.to_sql('Hotel', engine, if_exists="replace", index=False)

In [10]:
####
## create a dataframe to animate weather forecast
##################
df_cities  = pd.read_sql_table("City", engine)

cols_temps = ['Temp0','Temp1','Temp2','Temp3','Temp4','Temp5','Temp6']

list4anim=[]
for idx, row in df_cities.iterrows():
    day_count = 0
    for t in cols_temps:
        day = 'Day +{}'.format(day_count)
        list4anim.append({'City': row['City'], 'lat' : row['lat'], 'lon': row['lon'], 'Temperature':row[t], 'Day': day})
        day_count += 1

# Creates 7 records per cities used for animation
# should optimize this, one day ...
df4anim = pd.DataFrame(list4anim)

fig = px.scatter_mapbox(df4anim, lat="lat", lon="lon", 
    color="Temperature", size='Temperature', size_max=40, zoom=5, mapbox_style="carto-positron",
    hover_name='City', hover_data=['Temperature'],
    center = {'lat' : 46, 'lon' :2},
    color_continuous_scale=px.colors.sequential.Rainbow,
    animation_frame='Day')

fig.update_layout(
    hoverlabel=dict(
        bgcolor="white",
    )
)

fig.show()

2022-01-27 09:22:03,559 INFO sqlalchemy.engine.Engine SELECT c.relname FROM pg_class c JOIN pg_namespace n ON n.oid = c.relnamespace WHERE n.nspname = %(schema)s AND c.relkind in ('r', 'p')
2022-01-27 09:22:03,560 INFO sqlalchemy.engine.Engine [cached since 460s ago] {'schema': 'public'}
2022-01-27 09:22:03,572 INFO sqlalchemy.engine.Engine SELECT c.relname FROM pg_class c JOIN pg_namespace n ON n.oid = c.relnamespace WHERE n.nspname = %(schema)s AND c.relkind IN ('v', 'm')
2022-01-27 09:22:03,573 INFO sqlalchemy.engine.Engine [cached since 439.6s ago] {'schema': 'public'}
2022-01-27 09:22:03,580 INFO sqlalchemy.engine.Engine 
            SELECT c.oid
            FROM pg_catalog.pg_class c
            LEFT JOIN pg_catalog.pg_namespace n ON n.oid = c.relnamespace
            WHERE (pg_catalog.pg_table_is_visible(c.oid))
            AND c.relname = %(table_name)s AND c.relkind in
            ('r', 'v', 'm', 'f', 'p')
        
2022-01-27 09:22:03,580 INFO sqlalchemy.engine.Engine [cached 

In [ ]:
######
# prepare data to be displayed
###########
df_hotels = pd.read_sql_table("Hotel", engine)

df_hotels.desc = df_hotels.desc.str.wrap(30)
df_hotels.desc = df_hotels.desc.apply(lambda x: x.replace('\n', '<br>'))

fig = px.scatter_mapbox(df_hotels, lat='lat', lon='lon', 
    color='note', size='note', zoom=5, mapbox_style='carto-positron',
    color_continuous_scale='earth',
    center = {'lat' : 46, 'lon' :2},
    hover_name='name',
    hover_data=['name', 'note', 'href', 'desc'])

#print("plotly express hovertemplate:", fig.data[0].hovertemplate)

fig.update_traces(
    hovertemplate =
    '<b>%{customdata[0]}</b>' +
    '<br>'+
    '<b>Note: %{marker.color}</b>'+
    '<br>'+
    '<a href="%{customdata[2]}"><b>Reserver</b></a>'+
    '<br>'+
    '<a>%{customdata[3]}</a>'
    )

fig.update_layout(
    hoverlabel=dict(
        bgcolor="white",
    )
)

fig.show()